In [17]:
from scipy.stats import norm
import pandas as pd
import numpy as np
import datetime

#see also http://www.codeandfinance.com/finding-implied-vol.html

def find_vol(target_value, call_put, S, K, tau, r):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, tau, r, sigma)
        vega = bs_vega(call_put, S, K, tau, r, sigma)

        price = price
        diff = target_value - price  # f(x)

        #print (i, sigma, diff)

        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)

    # value wasn't found, return best guess so far
    return sigma
n = norm.pdf
N = norm.cdf

def bs_price(cp_flag,S,K,tau,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*tau)/(v*np.sqrt(tau))
    d2 = d1-v*np.sqrt(tau)
    if cp_flag == 'c':
        price = S*np.exp(-q*tau)*N(d1)-K*np.exp(-r*tau)*N(d2)
    else:
        price = K*np.exp(-r*tau)*N(-d2)-S*np.exp(-q*tau)*N(-d1)
    return price

def bs_vega(cp_flag,S,K,tau,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(tau))
    return S * np.sqrt(tau)*n(d1)

V_market = 17.5 #market value
K = 585 #strike price
start_date=datetime.datetime(2019,1,1) #start date of the option
end_date=datetime.datetime(2019,3,24)  #end date of the option
tau = (end_date - start_date).days / 365.  #time to maturity
S = 586.08 #asset price
r = 0.0002 #risk free rate
cp = 'c' # call option

implied_vol = find_vol(V_market, cp, S, K, tau, r)

print ('Implied vol: %.2f%%' % (implied_vol * 100))

print ('Market price = %.2f' % V_market)
print ('Model price = %.2f' % bs_price(cp, S, K, tau, r, implied_vol))

Implied vol: 15.30%
Market price = 17.50
Model price = 17.50
